## 결측치처리방식
### 1. 해당 노트북
+ 전처리방법2-3 + y 결측치를 예측하기

In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

### 전처리방법2 데이터 가져오기

In [35]:
train = pd.read_parquet('./data/train_preprocess_2.parquet')
# test = pd.read_parquet('./test.parquet')
test = pd.read_parquet('./data/test_preprocess_2.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

print(train.shape)
print(train.Delay.value_counts())

(1000000, 19)
Not_Delayed    210001
Delayed         45000
Name: Delay, dtype: int64


### 추가 결측치 처리

In [36]:
# 레이블(Delay)을 제외한 결측값이 존재하는 변수들을 학습 데이터의 최빈값으로 대체합니다
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

Done.


In [37]:
# 질적 변수들을 수치화합니다
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: # train에 없는 label인 경우
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.


In [38]:
train.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time         0
Estimated_Arrival_Time           0
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)               0
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
dtype: int64

### y값 결측 추출

In [39]:
# 레이블이 없는 데이터들을 따로 추출합니다.
semi_super_train = train[train['Delay'].isnull()]
semi_super_train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,600.0,1900.0,0,0,252,13851,34,159,12191,42,419.0,23,10,19393.0,4319,None
1,TRAIN_000001,8,15,740.0,1024.0,0,0,256,13930,11,331,14869,45,1250.0,22,8,20304.0,310,None
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,74,11057,31,204,12953,30,544.0,3,0,19805.0,140,None
3,TRAIN_000003,7,10,905.0,1735.0,0,0,195,12892,4,119,11618,28,2454.0,26,8,19977.0,3021,None
4,TRAIN_000004,1,11,900.0,1019.0,0,0,322,14771,4,7,10157,4,250.0,22,8,20304.0,556,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,256,13930,11,270,14100,36,678.0,26,8,19977.0,2477,None
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,122,11637,32,242,13487,21,223.0,22,3,20304.0,2294,None
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,248,13796,4,159,12191,42,1642.0,23,10,19393.0,994,None
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,45,10693,41,22,10397,8,214.0,9,3,19790.0,6207,None


In [40]:
# 레이블이 없는 데이터들을 삭제합니다.
train = train.dropna()
print(train.shape)

# Delay -> 1, Not_Delayed -> 0
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

train.head()

(255001, 19)
Done.


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
5,TRAIN_000005,4,13,1545.0,1900.0,0,0,119,11618,28,93,11278,47,199.0,21,8,20452.0,3435,Not_Delayed,0
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,119,11618,28,47,10721,19,200.0,26,8,19977.0,3495,Not_Delayed,0
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,59,10821,18,74,11057,31,361.0,23,10,19393.0,4083,Not_Delayed,0
10,TRAIN_000010,8,13,1730.0,1844.0,0,0,93,11278,47,277,14122,36,204.0,21,0,20452.0,241,Delayed,1
12,TRAIN_000012,1,12,1015.0,1145.0,0,0,72,11042,33,94,11292,5,1201.0,23,10,19393.0,5171,Not_Delayed,0


### y값 예측모델 구축1 - y값 결측치 처리
+ lgbm 이용해서 y값 예측
    + 기존에 좋은 예측 정확도가 나왔던 {'learning_rate': 0.075, 'max_depth': 8, 'n_estimators': 400, 'num_leaves': 35} 모형 사용

In [41]:
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
semi_super_train_x = semi_super_train.drop(columns=['ID','Delay'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42, learning_rate=0.075, max_depth=8, n_estimators=400, num_leaves=35)

# 모델 fit
lgbm.fit(train_x, train_y)

# 예측
pred_y = lgbm.predict_proba(semi_super_train_x)
pred_y # [Not Delayed, Delayed]


array([[0.7819333 , 0.2180667 ],
       [0.8531719 , 0.1468281 ],
       [0.77597428, 0.22402572],
       ...,
       [0.86328881, 0.13671119],
       [0.87790032, 0.12209968],
       [0.8881051 , 0.1118949 ]])

In [42]:
# 예측값 중 max index 찾아 semi_super_train의 y값으로 넣어주기
max_prob_idx = np.argmax(pred_y, axis=1)
print(max_prob_idx)
from collections import Counter
print('불연착/연착 분포 : ',Counter(max_prob_idx))
semi_super_train['Delay_num'] = max_prob_idx
semi_super_train

[0 0 0 ... 0 0 0]
불연착/연착 분포 :  Counter({0: 739961, 1: 5038})


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
0,TRAIN_000000,4,15,600.0,1900.0,0,0,252,13851,34,159,12191,42,419.0,23,10,19393.0,4319,None,0
1,TRAIN_000001,8,15,740.0,1024.0,0,0,256,13930,11,331,14869,45,1250.0,22,8,20304.0,310,None,0
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,74,11057,31,204,12953,30,544.0,3,0,19805.0,140,None,0
3,TRAIN_000003,7,10,905.0,1735.0,0,0,195,12892,4,119,11618,28,2454.0,26,8,19977.0,3021,None,0
4,TRAIN_000004,1,11,900.0,1019.0,0,0,322,14771,4,7,10157,4,250.0,22,8,20304.0,556,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,256,13930,11,270,14100,36,678.0,26,8,19977.0,2477,None,0
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,122,11637,32,242,13487,21,223.0,22,3,20304.0,2294,None,0
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,248,13796,4,159,12191,42,1642.0,23,10,19393.0,994,None,0
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,45,10693,41,22,10397,8,214.0,9,3,19790.0,6207,None,0


In [43]:
# 데이터 합치기
train = pd.concat([train, semi_super_train])
train.sort_index(inplace=True)
train


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
0,TRAIN_000000,4,15,600.0,1900.0,0,0,252,13851,34,159,12191,42,419.0,23,10,19393.0,4319,None,0
1,TRAIN_000001,8,15,740.0,1024.0,0,0,256,13930,11,331,14869,45,1250.0,22,8,20304.0,310,None,0
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,74,11057,31,204,12953,30,544.0,3,0,19805.0,140,None,0
3,TRAIN_000003,7,10,905.0,1735.0,0,0,195,12892,4,119,11618,28,2454.0,26,8,19977.0,3021,None,0
4,TRAIN_000004,1,11,900.0,1019.0,0,0,322,14771,4,7,10157,4,250.0,22,8,20304.0,556,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,256,13930,11,270,14100,36,678.0,26,8,19977.0,2477,None,0
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,122,11637,32,242,13487,21,223.0,22,3,20304.0,2294,None,0
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,248,13796,4,159,12191,42,1642.0,23,10,19393.0,994,None,0
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,45,10693,41,22,10397,8,214.0,9,3,19790.0,6207,None,0


### 전처리방법3 저장

In [44]:
save_idx = input('몇 번째 전처리 방법인지 정수-정수를 입력하세요 : ')
train_save_name = 'train_preprocess_' + save_idx
test_save_name = 'test_preprocess_' + save_idx
train.to_parquet(f'./data/{train_save_name}.parquet')
test.to_parquet(f'./data/{test_save_name}.parquet')

몇 번째 전처리 방법인지 정수-정수를 입력하세요 : 3
